In [31]:
import pandas as pd
import numpy as np
import requests
import json
import requests # buat download web
from tqdm import tqdm # progress bar
from bs4 import BeautifulSoup # filter html
import re # pattern matching
import ast

# Proses data scraping
Untuk mendapatkan data enrollment dan stars terkini karena enrollment dan stars nya belum tersedia

In [32]:
df = pd.read_csv("Downloads/Udemy_Courses.csv").query("Category=='Tech'") # Data list course udemy didapat dari kaggle, dan ini hanya memuat course yang dilaunch hingga tahun lalu.

In [33]:
df

,Title,Summary,Enrollment,Stars,Rating,Link,Category
0,2020 Complete Python Bootcamp: From Zero to He...,Learn Python like a Professional! Start from t...,1100093,4.6,318066,https://www.udemy.com/course/complete-python-b...,Tech
1,The Web Developer Bootcamp,The only course you need to learn web developm...,596726,4.6,182997,https://www.udemy.com/course/the-web-developer...,Tech
2,Machine Learning A-Z™: Hands-On Python & R In ...,Learn to create Machine Learning Algorithms in...,692812,4.5,132228,https://www.udemy.com/course/machinelearning/,Tech
3,Angular - The Complete Guide (2020 Edition),"Master Angular 10 (formerly ""Angular 2"") and b...",422557,4.6,129984,https://www.udemy.com/course/the-complete-guid...,Tech
4,Java Programming Masterclass for Software Deve...,Learn Java In This Course And Become a Compute...,502572,4.6,123798,https://www.udemy.com/course/java-the-complete...,Tech
...,...,...,...,...,...,...,...
9959,MonoDevelop/Visual Studio Mac & Unity ; Pro Ti...,Lay down consistent code at the speed of thoug...,84,4.0,9,https://www.udemy.com/course/monodevelop-with-...,Tech
9960,Unity Game Tutorial: Tetris 3D,Learn how to create a Tetris 3D in Unity Game ...,86,4.4,9,https://www.udemy.com/course/unity-game-tutori...,Tech
9961,Spring 5.0 Project: Building a Travel Website,Design and implement real-world web-based appl...,70,3.5,9,https://www.udemy.com/course/spring-50-project...,Tech
9962,Guide to Powershell 6 and Automating Active Di...,Use PowerShell for Active Directory to elimina...,119,4.2,9,https://www.udemy.com/course/guide-to-powershe...,Tech


In [34]:
# Diasumsikan course data science apabila pada title atau summary mengandung salah satu keyword dibawah
list_ds_keywords = ["machine learning","artificial intelligence","data scientist","statistics","tableau","pandas","tidyverse","tensorflow","pytorch","nlp","natural language processing","computer vision"] 

In [35]:
ds_related = (df['Title'] + " "+ df['Summary']).str.contains("|".join(list_ds_keywords),case =False)

df_ds_only = df[ds_related].reset_index().drop_duplicates(subset=["Link"])

df_ds_only

,index,Title,Summary,Enrollment,Stars,Rating,Link,Category
0,2,Machine Learning A-Z™: Hands-On Python & R In ...,Learn to create Machine Learning Algorithms in...,692812,4.5,132228,https://www.udemy.com/course/machinelearning/,Tech
1,7,Python for Data Science and Machine Learning B...,"Learn how to use NumPy, Pandas, Seaborn , Matp...",387789,4.6,87403,https://www.udemy.com/course/python-for-data-s...,Tech
2,10,The Data Science Course 2020: Complete Data Sc...,"Complete Data Science Training: Mathematics, S...",325047,4.5,76907,https://www.udemy.com/course/the-data-science-...,Tech
3,25,The Complete Web Developer in 2020: Zero to Ma...,Learn to code and become a Web Developer in 20...,138822,4.7,40976,https://www.udemy.com/course/the-complete-web-...,Tech
4,32,Deep Learning A-Z™: Hands-On Artificial Neural...,Learn to create Deep Learning Algorithms in Py...,278698,4.5,33454,https://www.udemy.com/course/deeplearning/,Tech
...,...,...,...,...,...,...,...,...
698,9785,Become a Visualization Expert Using TABLEAU,Learn Most popular Visualization tool Tableau,200,3.3,10,https://www.udemy.com/course/become-a-visualiz...,Tech
699,9802,Python for Computer Vision: Object (Traffic Si...,Learn how to develop an object detection appli...,53,3.9,10,https://www.udemy.com/course/python-for-comput...,Tech
701,9818,"AWS IoT:Building cool apps,AI& serverless Comp...",AWS Dl lets you use the latest Artificial Inte...,931,2.8,10,https://www.udemy.com/course/building-cool-app...,Tech
702,9829,neural networks for autoencoders and recommend...,How to build autoencoders and recommender syst...,100,4.3,9,https://www.udemy.com/course/neural-networks-f...,Tech


In [9]:
# Web scraping untuk mengambil info course terkini
list_save = []
for idx,url in enumerate(tqdm(df_ds_only['Link'].tolist())):
    try:
        search_page = requests.get(url) #download html
        search_soup = BeautifulSoup(search_page.content, 'html.parser') # object bs supaya gampang filter
        
        try:
            clip_course_id = search_soup.find("body").get('data-clp-course-id')
            parse_date = requests.get(f"https://www.udemy.com/api-2.0/courses/{clip_course_id}/?fields[course]=created").content
            date_created = ast.literal_eval(parse_date.decode('utf-8'))['created']
        except :
            date_created = -1
        
        num_students = search_soup.find("div", {"data-purpose":"enrollment"}).contents[0].strip()

        temp = search_soup.find("div", {"class":"ud-component--course-landing-page-udlite--sidebar-container"}).get('data-component-props')
        content_length = json.loads(temp)['componentProps']['incentives']['video_content_length']

        star_ratings = search_soup.find("span",{"class":re.compile("udlite-sr-only")}).contents[0]
        list_save.append([num_students,content_length,star_ratings,date_created])
    except : 
        list_save.append([-1,-1,-1,-1])

100%|████████████████████████████████████████████████████████████████████████████████| 674/674 [42:15<00:00,  3.76s/it]


In [10]:
df_save = pd.DataFrame(list_save,columns=["Peserta","Durasi","Rating","CreatedDate"])

df_save

,Peserta,Durasi,Rating,CreatedDate
0,807.315 peserta,44 jam,"Peringkat: 4,5 dari 5",2016-09-05T09:54:22Z
1,479.342 peserta,25 jam,"Peringkat: 4,6 dari 5",2016-07-13T05:22:58Z
2,427.388 peserta,"28,5 jam","Peringkat: 4,6 dari 5",2018-06-18T12:30:08Z
3,168.362 peserta,"35,5 jam","Peringkat: 4,7 dari 5",2017-11-12T17:16:11Z
4,314.461 peserta,"22,5 jam","Peringkat: 4,6 dari 5",2017-03-20T13:49:45Z
...,...,...,...,...
669,200 peserta,"8,5 jam","Peringkat: 3,3 dari 5",2016-08-11T06:58:00Z
670,78 peserta,2 jam,"Peringkat: 3,6 dari 5",2019-03-13T13:37:41Z
671,942 peserta,2 jam,"Peringkat: 2,8 dari 5",2018-07-23T13:34:01Z
672,154 peserta,2 jam,"Peringkat: 3,7 dari 5",2019-01-19T18:26:59Z


In [27]:
df_ds_only['count_students'] = df_save['Peserta'].str.split(" ").str[0].str.replace("\.","").astype(float).to_numpy()
df_ds_only['course_length'] = df_save['Durasi'].str.split(" ").str[0].str.replace("\,",".").replace({"":np.nan}).astype(float).to_numpy()
df_ds_only['course_rating']  = df_save['Rating'].str.split(" ").str[1].str.replace("\,",".").replace({"":np.nan}).astype(float).to_numpy()
df_ds_only['days_since_released'] = (date.today().today() - pd.to_datetime(df_save['CreatedDate']).dt.date).dt.days

df_ds_only.replace({-1:np.nan},inplace=True)

df_ds_only['count_students']= df_ds_only['count_students'].fillna(df_ds_only['Enrollment'])
df_ds_only['course_rating'] = df_ds_only['course_rating'].fillna(df_ds_only['Stars'])

df_ds_only.to_csv('udemy_temp.csv')

# Analysis

In [40]:
df_ds_only = pd.read_csv("udemy_temp.csv")
df_ds_only = df_ds_only.query('course_length > 10') # Kita bandingkan dengan course durasi tinggi juga.

df_ds_only

,index,Title,Summary,Enrollment,Stars,Rating,Link,Category,count_students,course_length,course_rating,days_since_released,estimated_profit,estimated_profit_per_month($)
0,2,Machine Learning A-Z™: Hands-On Python & R In ...,Learn to create Machine Learning Algorithms in...,692812,4.5,132228,https://www.udemy.com/course/machinelearning/,Tech,807315.0,44.0,4.5,1844.0,13724355.0,223281.263557
1,7,Python for Data Science and Machine Learning B...,"Learn how to use NumPy, Pandas, Seaborn , Matp...",387789,4.6,87403,https://www.udemy.com/course/python-for-data-s...,Tech,479342.0,25.0,4.6,1898.0,8148814.0,128801.064278
2,10,The Data Science Course 2020: Complete Data Sc...,"Complete Data Science Training: Mathematics, S...",325047,4.5,76907,https://www.udemy.com/course/the-data-science-...,Tech,427388.0,28.5,4.6,1193.0,7265596.0,182705.683152
3,25,The Complete Web Developer in 2020: Zero to Ma...,Learn to code and become a Web Developer in 20...,138822,4.7,40976,https://www.udemy.com/course/the-complete-web-...,Tech,168362.0,35.5,4.7,1411.0,2862154.0,60853.734940
4,32,Deep Learning A-Z™: Hands-On Artificial Neural...,Learn to create Deep Learning Algorithms in Py...,278698,4.5,33454,https://www.udemy.com/course/deeplearning/,Tech,314461.0,22.5,4.6,1648.0,5345837.0,97314.993932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,9659,"Read, write, and analyze data with pandas & Py...",Learn to code in Python and analyze data using...,121,3.3,10,https://www.udemy.com/course/datasciencepython/,Tech,128.0,25.0,3.3,NaN,2176.0,NaN
659,9673,The #1 Python Data Scientist: Sentiment Analys...,"Build Projects with Machine Learning, Text Cla...",49,4.6,10,https://www.udemy.com/course/the-1-python-data...,Tech,120.0,16.5,4.6,NaN,2040.0,NaN
662,9694,Create Arrays in Python NumPy - Learn Scientif...,Complete beginner's bootcamp for learning to c...,117,3.4,10,https://www.udemy.com/course/pythonnumpymath/,Tech,122.0,22.0,3.9,NaN,2074.0,NaN
665,9713,The Complete Machine Learning and Data Science...,Become a real time professional Data Scientist...,69,4.4,10,https://www.udemy.com/course/the-complete-mach...,Tech,179.0,17.5,4.8,NaN,3043.0,NaN


Sistem profit sharing udemy  
Jika student beli course karena ketemu sendiri di web udemy, maka udemy kasih 37% profit ke instruktur.  
Jika student beli course dari code referal instrukturnya, maka instrukturnya dapat 97%

In [41]:
# Asumsi perhitungan :
# Harga course adalah 25 dollar
# 80% student kita karena ketemu di website tanpa code referal kita, sisa 20% dari hasil marketing kita sendiri
df_ds_only['estimated_profit'] = df_ds_only['count_students']*0.8*25*0.37 + df_ds_only['count_students']*0.2*25*0.97

In [42]:
df_ds_only['estimated_profit_per_month($)'] = df_ds_only['estimated_profit']/df_ds_only['days_since_released']*30
df_ds_only.dropna()['estimated_profit_per_month($)'].describe().astype(str)

count                 191.0
mean      9975.548999641613
std      28061.499989244636
min      3.3455065100031756
25%       380.0377163827725
50%      2267.6684210526314
75%       6874.816476881137
max      265113.19042316254
Name: estimated_profit_per_month($), dtype: object

# Daftar top kompetitor

In [53]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
df_ds_only.sort_values('count_students',ascending=False).head(15)[['Title','Link','count_students','Stars','course_length','days_since_released']]

<ipython-input-53-c9b1a1f61a35>:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Title,Link,count_students,Stars,course_length,days_since_released
0,Machine Learning A-Z™: Hands-On Python & R In Data Science,https://www.udemy.com/course/machinelearning/,807315.0,4.5,44.0,1844.0
1,Python for Data Science and Machine Learning Bootcamp,https://www.udemy.com/course/python-for-data-science-and-machine-learning-bootcamp/,479342.0,4.6,25.0,1898.0
2,The Data Science Course 2020: Complete Data Science Bootcamp,https://www.udemy.com/course/the-data-science-course-complete-data-science-bootcamp/,427388.0,4.5,28.5,1193.0
104,Machine Learning & Deep Learning in Python & R,https://www.udemy.com/course/data_science_a_to_z/,323907.0,4.3,35.0,449.0
4,Deep Learning A-Z™: Hands-On Artificial Neural Networks,https://www.udemy.com/course/deeplearning/,314461.0,4.5,22.5,1648.0
92,Complete Machine Learning with R Studio - ML for 2020,https://www.udemy.com/course/machine-learning-with-r-studio/,220231.0,4.2,13.0,775.0
5,Data Science A-Z™: Real-Life Data Science Exercises Included,https://www.udemy.com/course/datascience/,193563.0,4.6,21.0,2310.0
3,The Complete Web Developer in 2020: Zero to Mastery,https://www.udemy.com/course/the-complete-web-developer-zero-to-mastery/,168362.0,4.7,35.5,1411.0
9,Artificial Intelligence A-Z™: Learn How To Build An AI,https://www.udemy.com/course/artificial-intelligence-az/,164974.0,4.3,16.5,1590.0
6,"Machine Learning, Data Science and Deep Learning with Python",https://www.udemy.com/course/data-science-and-machine-learning-with-python-hands-on/,153960.0,4.5,15.5,2138.0
